Pytorch 공부 사이트 : https://wikidocs.net/52460  

 1. 목차  
    1) 다중변수 선형회귀(Multivariate Linear Regression) 복습  
    2) "minibatch" Gradient Descent 이론  
    3) PyTorch Dataset and DataLoader : 데이터 정의(Data definition) 간단히  
 2. "minibatch" Gradient Descent : 엄청난 양의 데이터 다룸  
    0) 특징
        - 목적 : 복잡한 모델 문제 해결을 위함  
            기존 gradient descent :  
                엄청난 양의 데이터 필요  
                느린 속도  
                하드웨어적 한계  
            minibatch gradient descent : 일부분의 데이터만 학습하자!  
<img src="./Minibatch_Gradient_Descent.png" width="300" height="300">  
<img src="./Minibatch_Gradient_Descent2.png" width="400" height="400">  
        - 장점 : 업데이트 빠름  
        - 단점 : 잘못된 방향 업데이트 가능성  

In [1]:
# 전체 코드

# 모듈 불러오기
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy
import torch

import torch.nn as nn
import torch.nn.functional as F

# 데이터셋 상속
class CustomDataset(Dataset):
    # 데이터셋 정의 : __init__(self)
    def __init__(self):
        self.x_data = [[73, 80, 75], [93, 88, 93], [89, 91, 90], [96, 98, 100], [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

    # 데이터셋 원소 개수 : __len__(self)
    def __len__(self):
        return len(self.x_data)

    # 인덱스에 상응하는 데이터셋 데이터 : __getitem__(self, idx)
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

# 데이터셋 정의
dataset = CustomDataset()

# 데이터셋 설정
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)
    # 사용할 데이터셋
    # batch_size : minibatch의 크기
        # 보통 2의 제곱수 사용 ex. 16, 32, 64, 128, 256, 512, ...
    # shuffle : Epoch마다 데이터셋 섞어줌
    
# 모델 초기화
# W = torch.zeros((3, 1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 다중 선형 회귀이므로 input_dim=3, output_dim=1.

    def forward(self, x):
        return self.linear(x)

model = MultivariateLinearRegressionModel()
    # model = nn.Linear(3,1)
    
# Optimzer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader): # ♣
        x_train, y_train = samples
        # H(x) 계산
        # hypothesis = x_train.matmul(W) + b
        prediction = model(x_train)
        
        # cost 계산
        # cost = torch.mean((hypothesis - y_train) ** 2)
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{}, Batch : {}/{}, Cost : {:.6f}'.format(epoch, nb_epochs, batch_idx + 1, 
            len(dataloader), cost.item()))

Epoch    0/20, Batch : 1/3, Cost : 19461.726562
Epoch    0/20, Batch : 2/3, Cost : 3543.492920
Epoch    0/20, Batch : 3/3, Cost : 924.778503
Epoch    1/20, Batch : 1/3, Cost : 415.731049
Epoch    1/20, Batch : 2/3, Cost : 309.799774
Epoch    1/20, Batch : 3/3, Cost : 76.175262
Epoch    2/20, Batch : 1/3, Cost : 9.754256
Epoch    2/20, Batch : 2/3, Cost : 17.440784
Epoch    2/20, Batch : 3/3, Cost : 2.478501
Epoch    3/20, Batch : 1/3, Cost : 5.674511
Epoch    3/20, Batch : 2/3, Cost : 3.290919
Epoch    3/20, Batch : 3/3, Cost : 1.989660
Epoch    4/20, Batch : 1/3, Cost : 4.857816
Epoch    4/20, Batch : 2/3, Cost : 2.860577
Epoch    4/20, Batch : 3/3, Cost : 2.369841
Epoch    5/20, Batch : 1/3, Cost : 7.257009
Epoch    5/20, Batch : 2/3, Cost : 4.822524
Epoch    5/20, Batch : 3/3, Cost : 0.000306
Epoch    6/20, Batch : 1/3, Cost : 4.370424
Epoch    6/20, Batch : 2/3, Cost : 0.652697
Epoch    6/20, Batch : 3/3, Cost : 7.350837
Epoch    7/20, Batch : 1/3, Cost : 2.190479
Epoch    7/20, Ba